In [1]:
!pip install apache-beam

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import apache_beam as beam
import time
import random

# Der Tank
Als Eingabe erhalten wir eine Zahl, die als ID für den jeweiligen Liter Diesel dient.
Diese ID verwenden wir, um daraus einen Liter Diesel zu erzeugen – dargestellt als dict mit sechs Feldern:
`id`, `kind`, `amount`, `dirt`, `water` und `pressure`.
Der Treibstoff im Tank ist dort und erfährt nur atmosphärischen Druck. Ebenfalls kann er schmutz und wasser enthalten.

In [3]:
class Tank(beam.DoFn):
    def process(self, idx:int):
        e = {'id': idx, 'kind': 'diesel', 'amount': 1.0, 'dirt':random.uniform(0.01, 0.1), 'water':random.uniform(0.01, 0.1), 'pressure':0.0}
        print(f"🛢️  Tank              | id={idx:03d} | amt=1.0 L")
        yield e

# Elektrische Pumpe
Diese Pumpe befindet sich direkt im Tank und sorgt für einen Dieselstrom mit niedrigem Druck.

In [4]:
class LiftPump(beam.DoFn):
    def process(self, fuel:dict):
        fuel['pressure'] = 5.0 # bar
        print(f"🔌  Lift pump         | id={fuel['id']:03d} | amt={fuel['amount']:.1f} L")
        yield fuel

# Filter
Der Treibstoff durchläuft zwei Filter: Der erste entfernt Schmutzpartikel, der zweite trennt Wasser vom Diesel.

In [5]:
class Filter(beam.DoFn):
    def process(self, fuel):
        print(f"🧽  Filter            | id={fuel['id']:03d} | amt={fuel['amount']:.1f} L | dirt={fuel['dirt']*100:.1f}% | water={fuel['water']*100:.1f}%")

        fuel["dirt"] = 0.0 # filter dirt
        fuel["water"] = 0.0 # filter water
        yield fuel

# Mechanische Pumpe
Die mechanische Pumpe wird über den Zahnriemen direkt vom Motor angetrieben.
Sie verdichtet den Diesel auf sehr hohen Druck, damit der Kraftstoff über die Einspritzdüsen fein zerstäubt in die Brennkammer gelangen kann.

In [6]:
class HighPressurePump(beam.DoFn):
    def process(self, fuel):
        fuel['pressure'] += 1595.0 # bar
        print(f"⚙️  High-pressure     | id={fuel['id']:03d} | amt={fuel['amount']:.1f} L | pressure={fuel['pressure']} bar")
        yield fuel

# Der Motor

Der Motor besteht aus mehreren Komponenten, unter anderem der Common Rail und den Injektoren.
In Motor findet schlussendlich die Verbrennung des Luft–Treibstoff-Gemischs statt.

Da Dieselkraftstoff eine sehr niedrige Oktanzahl (25–30) besitzt, zündet er bereits bei rund 210 °C von selbst – also allein durch die Hitze der Kompression.
Benzin hingegen hat eine deutlich höhere Oktanzahl (95, 98 oder 100) und ist dadurch zündresistenter. Es benötigt eine Zündkerze, um das Gemisch zu entflammen.

Da der Motor Luft und Treibstoff ansaugt, diese selbstzündend verbrennt und am Ende Abgase ausstösst, modellieren wir diesen Prozess in der Pipeline als Transformer.
Dabei teilen wir jeden Liter Diesel nach der Verbrennung in fünf Teile Rauch, die den entstehenden Abgasen entsprechen.

In [7]:
class Engine(beam.PTransform):
    fuel_to_smoke_ratio = None
    def __init__(self, fuel_to_smoke_ratio=5.0):
        super().__init__()
        self.fuel_to_smoke_ratio = fuel_to_smoke_ratio

    def expand(self, pcoll):
        class CommonRail(beam.DoFn):
            def process(self, fuel):
                print(f"🟨  Common rail       | id={fuel['id']:03d} | amt={fuel['amount']:.1f}   | water={fuel['water']*100:.1f}% | dirt={fuel['dirt']*100:.1f}% |pressure={fuel['pressure']} bar | L")
                yield fuel

        class Air(beam.DoFn):
            def process(self, fuel):
                fuel['parts_air'] = 15 #Stoichiometry
                print(f"💨  Air               | id={fuel['id']:03d} | amt={fuel['amount']:.1f} L")
                yield fuel

        class Injectors(beam.DoFn):
            def process(self, fuel):
                print(f"💉  Injectors         | id={fuel['id']:03d} | amt={fuel['amount']:.1f} L")
                yield fuel

        class BurnFuel(beam.DoFn):
            fuel_to_smoke_ratio = None
            def __init__(self, fuel_to_smoke_ratio=5.0, *unused_args, **unused_kwargs):
                super().__init__(*unused_args, **unused_kwargs)
                self.fuel_to_smoke_ratio = fuel_to_smoke_ratio

            def process(self, fuel):
                time.sleep(0.2)
                part = fuel['amount'] / self.fuel_to_smoke_ratio
                for i in range(self.fuel_to_smoke_ratio):
                    smoke = {'id': fuel['id'], 'kind': 'smoke', 'amount': part, 'part': i+1}
                    print(f"🔥  Combustion        | id={fuel['id']:03d} | part={i+1}/{self.fuel_to_smoke_ratio} | amt={part:.1f} L | air={fuel['parts_air']}")
                    yield smoke

        class Turbo(beam.DoFn):
            def process(self, exhaustgas):
                print(f"🚀  Turbo             | id={exhaustgas['id']:03d}")
                yield exhaustgas

        return (pcoll
                | "CommonRail" >> beam.ParDo(CommonRail())
                | "PullAir" >> beam.ParDo(Air())
                | "Injectors" >> beam.ParDo(Injectors())
                | "Combustion" >> beam.ParDo(BurnFuel(fuel_to_smoke_ratio=self.fuel_to_smoke_ratio))
                | "Turbo" >> beam.ParDo(Turbo())

            )

# Abgasanlage
Schlussendlich strömen die Abgase vom Turbo und dem Wastegate durch die Abgasanlage und werden am Endstück in die Atmosphäre abgeführt.

In [8]:
class Exhaust(beam.DoFn):
    def process(self, exhaustgas):
        print(f"🌫️  Exhaust           | id={exhaustgas['id']:03d} | part={exhaustgas['part']}   | amt={exhaustgas['amount']:.1f} L")
        yield exhaustgas

# Komplettes System
Nun haben wir alle Teile gebaut und können alle mit Schläuchen und Rohren verbinden und unserem Fahrzeug Leben einhauchen.

In [9]:
tank_capacity = 75 # liters
fuel_to_smoke_ratio = 4

with beam.Pipeline() as p:
    tank = (
        p
        | "IDs(0..74)" >> beam.Create(range(tank_capacity))
        | "Tank" >> beam.ParDo(Tank())
    )

    fuel_flow = (
        tank
        | "LiftPump" >> beam.ParDo(LiftPump())
        | "Filter" >> beam.ParDo(Filter())
        | "HighPressure" >> beam.ParDo(HighPressurePump())
    )

    exhaust = (
        fuel_flow
        | "Engine" >> Engine(fuel_to_smoke_ratio=fuel_to_smoke_ratio)
    )
    (
        exhaust
        | "Exhaust" >> beam.ParDo(Exhaust())
        | "Amounts" >> beam.Map(lambda s: s['amount'])
        | "TotalSmoke" >> beam.CombineGlobally(sum)
        | "Summary" >> beam.Map(lambda total:
            print(f"\n✅ Summary: 75 L → {total:.1f} L across {total * fuel_to_smoke_ratio} smoke parts ({fuel_to_smoke_ratio} per liter).\n"))
    )

🛢️  Tank              | id=000 | amt=1.0 L
🔌  Lift pump         | id=000 | amt=1.0 L
🧽  Filter            | id=000 | amt=1.0 L | dirt=8.4% | water=8.8%
⚙️  High-pressure     | id=000 | amt=1.0 L | pressure=1600.0 bar
🟨  Common rail       | id=000 | amt=1.0   | water=0.0% | dirt=0.0% |pressure=1600.0 bar | L
💨  Air               | id=000 | amt=1.0 L
💉  Injectors         | id=000 | amt=1.0 L
🔥  Combustion        | id=000 | part=1/4 | amt=0.2 L | air=15
🚀  Turbo             | id=000
🌫️  Exhaust           | id=000 | part=1   | amt=0.2 L
🔥  Combustion        | id=000 | part=2/4 | amt=0.2 L | air=15
🚀  Turbo             | id=000
🌫️  Exhaust           | id=000 | part=2   | amt=0.2 L
🔥  Combustion        | id=000 | part=3/4 | amt=0.2 L | air=15
🚀  Turbo             | id=000
🌫️  Exhaust           | id=000 | part=3   | amt=0.2 L
🔥  Combustion        | id=000 | part=4/4 | amt=0.2 L | air=15
🚀  Turbo             | id=000
🌫️  Exhaust           | id=000 | part=4   | amt=0.2 L
🛢️  Tank              |